<a href="https://colab.research.google.com/github/haegomm/ai_practice/blob/master/%EB%91%90_%EB%AC%B8%EC%9E%A5%EC%9D%98_%EB%85%BC%EB%A6%AC%EC%A0%81_%EB%AA%A8%EC%88%9C_%EB%B6%84%EB%A5%98_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HuggingFace로 두 문장의 논리적 모순 분류

In [80]:
# 필요한 라이브러리 설치 및 임포트
# transformers: 사전 학습된 NLP 모델 사용
# datasets: 공개 데이터셋 로드
# evaluate: 평가 지표 계산
# numpy: 수치 계산
!pip install transformers datasets evaluate accelerate scikit-learn

In [81]:
import random
import evaluate
import numpy as np

# MNLI 데이터셋 로드
# premise와 hypothesis 두 문장이 주어졌을 때, 두 문장의 논리적 관계(entailment, neutral, contradiction)를 분류
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [82]:
# GLUE 벤치마크에서 MNLI 데이터셋 불러오기
imdb = load_dataset("nyu-mll/glue", "mnli")

In [95]:
# 데이터의 첫 번째 샘플 확인
imdb['train'][0]

{'premise': 'Conceptually cream skimming has two basic dimensions - product and geography.',
 'hypothesis': 'Product and geography are what make cream skimming work. ',
 'label': 1,
 'idx': 0}

In [84]:
# 사전 학습된 BERT 모델의 토크나이저를 로드
# bert-base-cased는 대소문자를 구분하며, BERT 모델 구조를 따름
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

# 데이터 전처리 함수 정의
# premise와 hypothesis를 입력으로 받아 토크나이징
def preprocess_function(data):
    return tokenizer(data["premise"], data["hypothesis"], truncation=True)

# 데이터셋에 전처리 적용 (batched=True로 여러 샘플을 한 번에 처리)
imdb_tokenized = imdb.map(preprocess_function, batched=True)

In [85]:
# 전처리된 데이터의 첫 번째 샘플에서 사용 가능한 키 확인
imdb_tokenized['train'][0].keys()

dict_keys(['premise', 'hypothesis', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'])

In [86]:
# 전처리된 데이터의 첫 번째 샘플 출력(궁금해서 출력해봄)
sample = imdb_tokenized['train'][0]

for key, value in sample.items():
  print(f"{key}: {value}")

# input_ids: 텍스트를 토큰화하여 각 토큰을 고유한 숫자(ID)로 변환한 결과
# token_type_ids: 문장 쌍(premise와 hypothesis)을 구분하는 데 사용
# attention_mask: 실제 문장의 부분은 1, 패딩된 부분(의미 없는 부분)은 0으로 설정

premise: Conceptually cream skimming has two basic dimensions - product and geography.
hypothesis: Product and geography are what make cream skimming work. 
label: 1
idx: 0
input_ids: [101, 28103, 14795, 7081, 10458, 25004, 1144, 1160, 3501, 10082, 118, 3317, 1105, 14534, 119, 102, 22249, 1105, 14534, 1132, 1184, 1294, 7081, 10458, 25004, 1250, 119, 102]
token_type_ids: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
attention_mask: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [87]:
# train 데이터셋에서 20%를 validation 데이터로 분리
# 학습용(train) 데이터와 검증용(validation) 데이터로 나누기
imdb_split = imdb_tokenized['train'].train_test_split(test_size=0.2)
imdb_train = imdb_split['train']
imdb_val = imdb_split['test']

# 데이터셋 구조 확인
print(imdb_split)

DatasetDict({
    train: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 314161
    })
    test: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 78541
    })
})


In [88]:
# 학습용 데이터와 검증용 데이터의 크기 확인
len(imdb_train), len(imdb_val)

(314161, 78541)

In [89]:
# BERT 모델 구성 설정
from transformers import BertConfig

# BERT 모델의 하이퍼파라미터를 커스터마이징
config = BertConfig()

config.hidden_size = 64  # BERT layer의 기본 hidden dimension
config.intermediate_size = 64  # FFN layer의 중간 hidden dimension
config.num_hidden_layers = 2  # BERT layer의 개수
config.num_attention_heads = 4  # Multi-head attention에서 사용하는 head 개수
config.num_labels = 3  # MNLI 문제는 3개의 클래스(entailment, neutral, contradiction)

# 위 설정을 기반으로 BERT 모델을 생성
model = AutoModelForSequenceClassification.from_config(config)

In [90]:
# 학습 설정
from transformers import TrainingArguments, Trainer

# TrainingArguments: 학습에 필요한 설정값을 정의
training_args = TrainingArguments(
    output_dir='./hf_mnli_model',     # 모델과 로그 저장 경로
    num_train_epochs=20,              # 학습 epoch 수
    per_device_train_batch_size=128,  # training data의 batch size
    per_device_eval_batch_size=128,   # validation data의 batch size
    logging_dir='./logs',             # 로그 저장 경로
    logging_strategy="epoch",         # Epoch가 끝날 때마다 training loss 등을 log하라는 의미
    do_train=True,                    # 학습을 진행하겠다는 의미
    do_eval=True,                     # 학습 중간에 validation data에 대한 평가를 수행하겠다는 의미
    eval_strategy="epoch",            # 매 epoch가 끝날 때마다 validation data에 대한 평가를 수행한다는 의미
    save_strategy="epoch",            # 매 epoch가 끝날 때마다 모델을 저장하겠다는 의미
    learning_rate=2e-5,               # optimizer에 사용할 learning rate
    load_best_model_at_end=True,      # 학습이 끝난 후, validation data에 대한 성능이 가장 좋은 모델을 채택하겠다는 의미
    report_to="none"                  # 외부 로깅 도구(W&B, TensorBoard 등)를 사용하지 않고, 로컬 로그만 기록하도록 설정
)

In [91]:
# 평가 지표 정의
import evaluate

accuracy = evaluate.load("accuracy")

# 평가 함수 정의
# 모델의 출력(predictions)과 실제 레이블(labels)을 비교해 정확도를 계산
def compute_metrics(pred):
    predictions, labels = pred
    predictions = np.argmax(predictions, axis=1) # 가장 높은 확률 값을 예측값으로 선택
    return accuracy.compute(predictions=predictions, references=labels)

In [92]:
from transformers import EarlyStoppingCallback

#mnli_train_limited = imdb_train.select(range(10000))

# Trainer 설정
# Trainer는 Hugging Face에서 제공하는 학습/평가를 쉽게 수행할 수 있는 도구
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=imdb_train,
    eval_dataset=imdb_val,            # 검증 데이터
    compute_metrics=compute_metrics,  # 평가 함수
    tokenizer=tokenizer,
    # callbacks = [EarlyStoppingCallback(early_stopping_patience=1)]
)

<ipython-input-92-d87eaa046efa>:5: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [93]:
# 모델 학습
# (정확도 50넘어서 멈춤)
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,1.085600,1.060770,0.426847
2,1.041000,1.028210,0.464624
3,1.012100,1.014424,0.479202
4,0.998800,1.011316,0.479138
5,0.989500,1.006081,0.484308
6,0.981400,1.002171,0.490890
7,0.972300,0.997867,0.498529
8,0.963000,0.995207,0.504386
9,0.956200,0.992101,0.508104
10,0.950100,0.992298,0.510485


KeyboardInterrupt: 

In [94]:
# imdb_val (train split에서 분리된 검증 데이터)로 평가
eval_result = trainer.evaluate(imdb_val)
print(f"Validation Accuracy: {eval_result['eval_accuracy']:.2f}")

Epoch,Training Loss,Validation Loss,Accuracy
1,1.085600,1.060770,0.426847
2,1.041000,1.028210,0.464624
3,1.012100,1.014424,0.479202
4,0.998800,1.011316,0.479138
5,0.989500,1.006081,0.484308
6,0.981400,1.002171,0.490890
7,0.972300,0.997867,0.498529
8,0.963000,0.995207,0.504386
9,0.956200,0.992101,0.508104
10,0.950100,0.992298,0.510485


Validation Accuracy: 0.52
